In [40]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import transforms

In [41]:
print (torch.__version__)

1.13.0


In [77]:
class NN(nn.Module):
    def __init__(self,input_size,num_classes):
        super(NN,self).__init__()
        self.fc1=nn.Linear(input_size,50)
        self.fc2=nn.Linear(50,num_classes)
    
    def forward(self,x):
        x=F.relu(self.fc1(x))
        x=self.fc2(x)
        return x

In [43]:
##Device
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [44]:
#Hyperparameters
input_size=784
num_classes=10
learning_rate=0.001
batch_size=64
num_epochs=1

In [45]:
##Load Data
train_dataset=datasets.MNIST(root='datasets/',train=True,transform=transforms.ToTensor(),download=True)
train_loader=DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True)
test_dataset=datasets.MNIST(root='datasets/',train=False,transform=transforms.ToTensor(),download=True)
test_loader=DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=True)

In [79]:
#Intialize the model
model=NN(input_size=input_size,num_classes=num_classes).to(device)

In [75]:
#Loss Function
criterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(),lr=learning_rate)

In [86]:
#train the model
for epoch in range(num_epochs):
    for batch_idx, (data,target) in enumerate(train_loader):
        #Set data to cuda
        data=data.to(device=device)
        target=target.to(device=device)
        #Reshape 
        data=data.reshape(data.shape[0],-1)
        #forward
        scores=model(data)
        #loss
        loss=criterion(scores,target)
        #Backward
        optimizer.zero_grad()
        loss.backward()
        #GD or adam step
        optimizer.step()

In [71]:
#Check accuracy
def check_accuracy(loader,model):
    if loader.dataset.train:
        print("Accuracy on train data: ")
    else:
        print("Accuracy on test data: ")
    num_correct=0
    num_samples=0
    model.eval()
    with torch.no_grad():
        for x,y in loader:
            x=x.to(device=device)
            y=y.to(device=device)
            x=x.reshape(x.shape[0],-1)
            score=model(x)
            _,pred=score.max(1)
            num_correct+=(pred==y).sum()
            num_samples+=pred.size(0)
    print(f'Got {num_correct} / {num_samples} with a accuracy of {float(num_correct)/float(num_samples)*100:.2f}')
    model.train()

In [72]:
check_accuracy(train_loader,model)
check_accuracy(test_loader,model)

Accuracy on train data: 
Got 55761 / 60000 with a accuracy of 92.94
Accuracy on test data: 
Got 9303 / 10000 with a accuracy of 93.03
